<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/news_RNN_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tbody><tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a rel="noopener" href="https://www.thelmbook.com">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a rel="noopener" href="https://github.com/aburkov/theLMbook">https://github.com/aburkov/theLMbook</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </tbody></table>
    </div>
</div>

# RNN 기반 언어 모델

## 유틸리티 함수와 클래스

다음 셀에서 필요 라이브러리를 임포트하고 유틸리티 함수와 모델 클래스를 정의합니다.

In [1]:
# 필수 라이브러리 임포트
import os               # 파일 및 경로 작업을 위한 라이브러리 (check_file_exists, extract_dataset)
import urllib.request   # URL에서 데이터셋 파일 다운로드를 위한 라이브러리
import tarfile          # .tar.gz 데이터셋 압축 해제를 위한 라이브러리
import torch            # 텐서 연산과 딥러닝을 위한 메인 파이토치 라이브러리
import torch.nn as nn   # 신경망 모듈, 층 및 유틸리티
from torch.utils.data import DataLoader, IterableDataset  # 효율적인 데이터 로딩 및 스트리밍을 위한 라이브러리
import random           # 재현성을 위한 난수 시드 설정
from tqdm import tqdm   # 훈련 및 평가 시 진행률 표시를 위한 라이브러리
import math             # exp()를 사용한 혼잡도 계산을 위한 라이브러리
import re               # 텍스트 전처리를 위한 라이브러리 (숫자를 자리 표시자로 대체)
from transformers import AutoTokenizer # 사전 훈련된 토크나이저 로드를 위한 라이브러리

# ----------------------------
# 유틸리티 함수
# ----------------------------
def set_seed(seed):
    """
    다양한 파이썬 라이브러리에서 재현성을 위해 난수 시드를 설정합니다.
    이렇게 하면 여러 실행에 걸쳐 랜덤한 연산이 동일한 결과를 제공합니다.
    Args:
        seed (int): 난수 생성을 위한 시드 값
    """
    # 파이썬 내장 random 모듈의 시드 설정
    random.seed(seed)
    # 파이토치의 CPU 난수 생성기 시드 설정
    torch.manual_seed(seed)
    # 파이토치의 GPU 난수 생성기 시드 설정
    torch.cuda.manual_seed_all(seed)
    # 가능한 경우 cuDNN이 결정론적 알고리즘을 사용하도록 요청
    # 참고: 이는 성능에 영향을 미칠 수 있으며 모든 경우에서 결정론을 보장하지 않을 수 있습니다
    torch.backends.cudnn.deterministic = True
    # 특정 입력 크기에 대해 최적의 알고리즘을 찾는 cuDNN의 자동 튜너를 비활성화
    # 일관된 동작을 보장하지만 입력 크기를 최적화하지 않으므로 더 느릴 수 있습니다
    torch.backends.cudnn.benchmark = False

class IterableTextDataset(IterableDataset):
    """
    메모리 효율적인 방식으로 텍스트 데이터를 처리하기 위한 반복 가능한 데이터셋입니다.
    모든 데이터를 메모리에 로드하는 대신 디스크에서 데이터를 스트리밍합니다.
    스트리밍 지원을 위해 파이토치의 IterableDataset을 상속합니다.
    Args:
        file_path (str): 문장이 포함된 텍스트 파일의 경로
        tokenizer: 텍스트를 토큰으로 변환하기 위한 토크나이저 객체
        max_length (int): 처리할 최대 시퀀스 길이 (기본값: 30)
    """
    def __init__(self, file_path, tokenizer, max_length=30):
        # 데이터 읽기를 위한 파일 경로 저장
        self.file_path = file_path
        # 텍스트 처리를 위한 토크나이저 저장
        self.tokenizer = tokenizer
        # 긴 시퀀스를 자르기 위한 최대 시퀀스 길이 설정
        self.max_length = max_length
        self._count_sentences()

    def __iter__(self):
        """
        데이터셋에 대한 반복자를 생성합니다.
        이 메서드는 데이터셋을 반복할 때 호출됩니다.
        Yields:
            tuple: 언어 모델링을 위한 (input_sequence, target_sequence) 쌍
                  input_sequence는 마지막 토큰까지의 시퀀스
                  target_sequence는 한 위치 오른쪽으로 이동한 시퀀스
        """
        # UTF-8 인코딩으로 읽기 모드에서 파일 열기
        with open(self.file_path, 'r', encoding="utf-8") as f:
            # 파일의 각 라인(문장) 처리
            for line in f:
                # 앞뒤 공백 제거
                sentence = line.strip()
                # 모든 숫자를 ### 자리 표시자로 대체
                # 이는 어휘 크기를 줄이고 모델의 일반화에 도움이 됩니다
                sentence = re.sub(r"\d+", "###", sentence)
                # 문장을 토큰 ID로 변환
                encoded_sentence = self.tokenizer.encode(
                    sentence,
                    max_length=self.max_length,
                    truncation=True
                )
                # 최소 2개의 토큰이 있는 시퀀스만 사용
                # (최소 하나의 입력과 하나의 타깃 토큰이 필요)
                if len(encoded_sentence) >= 2:
                    # 입력은 마지막을 제외한 모든 토큰
                    input_seq = encoded_sentence[:-1]
                    # 타깃은 첫 번째를 제외한 모든 토큰
                    target_seq = encoded_sentence[1:]
                    # 파이토치 텐서로 변환하여 반환
                    yield torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_seq, dtype=torch.long)

    def __len__(self):
        return self._num_sentences

    def _count_sentences(self):
        print(f"{self.file_path}에서 문장 수 계산 중...")
        with open(self.file_path, 'r', encoding="utf-8") as f:
            self._num_sentences = sum(1 for _ in f)
        print(f"{self.file_path}에서 {self._num_sentences}개의 문장을 찾았습니다.")

# ----------------------------
# 데이터 다운로드 및 준비
# ----------------------------
def create_collate_fn(tokenizer):
    """
    다양한 길이의 시퀀스를 배치로 만들기 위한 collate 함수를 생성합니다.
    이 함수는 더 짧은 시퀀스를 배치에서 가장 긴 시퀀스와 일치하도록 패딩합니다.
    Args:
        tokenizer: 패딩 토큰 정보를 포함하는 토크나이저 객체
    Returns:
        function: 배치에서 패딩을 처리하는 collate 함수
    """
    def collate_fn(batch):
        # 배치에서 입력과 타깃 분리
        input_seqs, target_seqs = zip(*batch)
        # 토크나이저에서 패딩 토큰 ID 가져오기
        pad_index = tokenizer.pad_token_id
        # 입력 시퀀스를 동일한 길이로 패딩
        input_padded = nn.utils.rnn.pad_sequence(input_seqs, batch_first=True, padding_value=pad_index)
        # 타깃 시퀀스를 동일한 길이로 패딩
        target_padded = nn.utils.rnn.pad_sequence(target_seqs, batch_first=True, padding_value=pad_index)
        return input_padded, target_padded
    return collate_fn

def check_file_exists(filename):
    """
    현재 디렉토리에 파일이 있는지 확인합니다.
    Args:
        filename (str): 확인할 파일 이름
    Returns:
        bool: 파일이 있으면 True, 없으면 False
    """
    return os.path.exists(filename)

def download_file(url):
    """
    로컬에 파일이 없는 경우 주어진 URL에서 파일을 다운로드합니다.
    다운로드 차단을 방지하기 위해 사용자 지정 User-Agent를 사용합니다.
    Args:
        url (str): 다운로드할 파일의 URL
    Returns:
        str: 다운로드된 파일의 이름 ("news.tar.gz")
    """
    # URL에 관계없이 항상 news.tar.gz를 파일 이름으로 사용
    filename = "news.tar.gz"
    if not check_file_exists(filename):
        print(f"{url}에서 데이터셋 다운로드 중...")
        req = urllib.request.Request(
            url,
            headers={"User-Agent": "Mozilla/5.0"}
        )
        with urllib.request.urlopen(req) as response:
            with open(filename, "wb") as out_file:
                out_file.write(response.read())
        print("다운로드가 완료되었습니다.")
    else:
        print(f"{filename}은(는) 이미 다운로드되었습니다.")
    return filename

def is_within_directory(directory, target):
    """
    절대 경로를 비교하여 대상 경로가 지정된 디렉토리 내에 있는지 확인합니다.
    Args:
        directory (str): 기본 디렉토리 경로
        target (str): 확인할 대상 경로
    Returns:
        bool: 대상의 절대 경로가 디렉토리의 절대 경로로 시작하면 True
    """
    abs_directory = os.path.abspath(directory)
    abs_target = os.path.abspath(target)
    prefix = os.path.commonprefix([abs_directory, abs_target])
    return prefix == abs_directory

def extract_dataset(filename):
    """
    다운로드한 아카이브에서 train.txt와 test.txt를 추출합니다.
    아카이브 내용에 대한 디버그 정보를 포함합니다.
    Args:
        filename (str): 아카이브 파일의 이름
    Returns:
        tuple: 추출된 train과 test 파일의 경로
    """
    data_dir = os.path.join(os.path.dirname(filename), "news")
    train_path = os.path.join(data_dir, "train.txt")
    test_path = os.path.join(data_dir, "test.txt")

    if check_file_exists(train_path) and check_file_exists(test_path):
        print("데이터 파일이 이미 추출되었습니다.")
        return train_path, test_path

    print("\n아카이브 내용 나열:")
    with tarfile.open(filename, "r:gz") as tar:
        for member in tar.getmembers():
            print(f"아카이브 멤버: {member.name}")
        print("\n파일 추출 중...")
        # 먼저 현재 디렉토리로 추출
        tar.extractall('.')

    if not (check_file_exists(train_path) and check_file_exists(test_path)):
        raise FileNotFoundError(f"아카이브에서 필요한 파일을 찾을 수 없습니다. 위의 경로를 확인하십시오.")

    print("추출이 완료되었습니다.")
    return train_path, test_path

def create_datasets(train_file, test_file, tokenizer, max_length=30):
    """
    훈련과 테스트를 위한 IterableTextDataset 객체를 생성합니다.
    이 데이터셋은 모든 데이터를 메모리에 로드하는 대신 디스크에서 데이터를 스트리밍합니다.
    Args:
        train_file (str): 훈련 데이터 파일의 경로
        test_file (str): 테스트 데이터 파일의 경로
        tokenizer: 텍스트 처리를 위한 토크나이저 객체
    Returns:
        tuple: (train_dataset, test_dataset) - 훈련과 테스트를 위한 데이터셋 객체
    """
    # 훈련 데이터셋 생성
    train_dataset = IterableTextDataset(train_file, tokenizer, max_length)
    # 테스트 데이터셋 생성
    test_dataset = IterableTextDataset(test_file, tokenizer, max_length)
    # 데이터셋 크기 출력
    print(f"훈련 문장 수: {len(train_dataset)}")
    print(f"테스트 문장 수: {len(test_dataset)}")
    return train_dataset, test_dataset

def create_dataloaders(train_dataset, test_dataset, batch_size, collate_fn):
    """
    효율적인 데이터 반복을 위한 DataLoader 객체를 생성합니다.
    Args:
        train_dataset: 훈련 데이터셋
        test_dataset: 테스트 데이터셋
        batch_size (int): 배치당 시퀀스 수
        collate_fn: 패딩과 배치 생성을 처리하는 함수
    Returns:
        tuple: (train_dataloader, test_dataloader) - 적절한 패딩으로
               데이터 배치를 반복하기 위한 DataLoader 객체
    """
    # 훈련 데이터 로더 생성
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,    # 패딩을 처리하는 함수
        num_workers=0             # 워커 프로세스 수 (0 = 단일 프로세스)
    )
    # 테스트 데이터 로더 생성
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        num_workers=0
    )
    return train_dataloader, test_dataloader

def download_and_prepare_data(url, batch_size, tokenizer, max_length=30):
    """
    전체 데이터 준비 파이프라인을 처리하는 메인 함수입니다.
    데이터를 다운로드하고, 추출하고, 필요한 데이터셋 객체를 생성합니다.
    Args:
        url (str): 데이터셋 아카이브를 다운로드할 수 있는 URL
        batch_size (int): 데이터 로딩을 위한 배치 크기
        tokenizer: 텍스트 처리를 위한 토크나이저 객체
        max_length (int): 토큰화를 위한 최대 시퀀스 길이 (기본값: 30)
    Returns:
        tuple: (train_dataloader, test_dataloader) - 사용 준비된 데이터 로더
    """
    # 1단계: URL에서 데이터셋 아카이브 다운로드
    filename = download_file(url)
    # 2단계: 아카이브에서 훈련 및 테스트 파일 추출
    train_file, test_file = extract_dataset(filename)
    # 3단계: 데이터 스트리밍을 위한 데이터셋 객체 생성
    train_dataset, test_dataset = create_datasets(train_file, test_file, tokenizer, max_length)
    # 4단계: 배치 생성을 처리하는 함수 생성
    collate_fn = create_collate_fn(tokenizer)
    # 5단계: 데이터 로더 생성 및 반환
    return create_dataloaders(train_dataset, test_dataset, batch_size, collate_fn)

def compute_loss_and_perplexity(model, dataloader, tokenizer, criterion, device, max_sentences=1000):
    """
    데이터에 대한 손실과 혼잡도를 계산하여 모델 성능을 평가합니다.
    Args:
        model (nn.Module): 평가할 언어 모델
        dataloader (DataLoader): 배치 시퀀스를 포함하는 데이터 로더
        tokenizer: 패딩과 같은 특수 토큰을 처리하기 위한 토크나이저
        criterion: 손실 함수 (보통 CrossEntropyLoss)
        device: 계산을 실행할 장치 (cuda/cpu)
        max_sentences (int): 평가할 최대 문장 수 (기본값: 1000)
                           더 빠른 검증을 위해 평가를 하위 집합으로 제한합니다
    Returns:
        tuple: (average_loss, perplexity)
               - average_loss: 토큰당 평균 손실 (패딩 제외)
               - perplexity: exp(average_loss), 낮을수록 좋음
    """
    # 모델을 평가 모드로 설정 (드롭아웃 등 비활성화)
    model.eval()
    # 손실 계산을 위한 카운터 초기화
    total_loss = 0.0          # 모든 배치에 대한 총 손실 누적 변수
    total_tokens = 0          # 총 토큰 수 카운터 (패딩 제외)
    sentences_processed = 0    # 처리된 문장 수 카운터

    # 효율성을 위해 그래디언트 계산 비활성화
    with torch.no_grad():
        # 진행률 표시와 함께 데이터 반복
        for input_seq, target_seq in tqdm(dataloader, desc="평가 중", leave=False):
            # 입력과 타깃 시퀀스를 지정된 장치로 이동
            input_seq = input_seq.to(device)      # 크기: (batch_size, seq_len)
            target_seq = target_seq.to(device)    # 크기: (batch_size, seq_len)

            # 현재 배치 크기 가져오기 (마지막 배치는 더 작을 수 있음)
            batch_size_current = input_seq.size(0)

            # 모델을 통한 포워드 패스
            logits = model(input_seq)             # 크기: (batch_size, seq_len, vocab_size)

            # 손실 계산을 위해 로짓과 타깃 재구성
            logits = logits.reshape(-1, logits.size(-1))  # 크기: (batch_size * seq_len, vocab_size)
            target = target_seq.reshape(-1)              # 크기: (batch_size * seq_len)

            # 패딩 토큰을 제외하는 마스크 생성
            mask = target != tokenizer.pad_token_id

            # 패딩되지 않은 토큰에 대해서만 손실 계산
            loss = criterion(logits[mask], target[mask])

            # 카운터 업데이트
            loss_value = loss.item() * mask.sum().item()  # 이 배치의 총 손실
            total_loss += loss_value                      # 배치 손실 누적
            total_tokens += mask.sum().item()             # 패딩이 아닌 토큰 수 계산

            # 문장 카운터 업데이트 및 최대값에 도달했는지 확인
            sentences_processed += batch_size_current
            if sentences_processed >= max_sentences:
                break

    # 최종 메트릭 계산
    average_loss = total_loss / total_tokens           # 토큰 수로 손실 정규화
    perplexity = math.exp(average_loss)               # 손실을 혼잡도로 변환
    return average_loss, perplexity

def perform_model_evaluation(model, test_dataloader, criterion, tokenizer, device, contexts):
    """
    손실 계산, 혼잡도 및 텍스트 생성을 포함한 모델 평가를 수행합니다.
    Args:
        model: 신경망 모델
        test_dataloader: 테스트/검증 데이터를 포함하는 DataLoader
        criterion: 손실 함수
        tokenizer: 텍스트 생성을 위한 토크나이저
        device: 계산을 실행할 장치 (cuda/cpu)
        contexts: 텍스트 생성을 위한 문맥 문자열 목록
    Returns:
        tuple: (average_loss, perplexity)
    """
    # 평가 모드로 전환
    model.eval()

    # 메트릭 계산
    average_loss, perplexity = compute_loss_and_perplexity(
        model, test_dataloader, tokenizer, criterion, device, max_sentences=1000
    )
    print(f"검증 평균 손실: {average_loss:.4f}, 혼잡도: {perplexity:.2f}")

    # 문맥을 사용한 텍스트 생성
    print("generate_text를 사용하여 문맥을 기반으로 텍스트 생성:\n")
    for context in contexts:
        generated_text = generate_text(
            model=model,          # 로드된 언어 모델
            start_string=context, # 시작 문맥
            tokenizer=tokenizer,  # 텍스트 변환을 위한 토크나이저
            device=device,        # CPU 또는 GPU 장치
            max_length=50         # 생성된 시퀀스의 최대 길이
        )
        print(f"\n문맥: {context}")
        print(f"\n생성된 텍스트: {generated_text}\n")

    return average_loss, perplexity

def generate_text(model, start_string, tokenizer, device, max_length=50):
    """
    탐욕적 디코딩을 사용하여 주어진 시작 문자열에서 텍스트를 이어갑니다.
    이 메서드는 항상 가장 가능성 있는 다음 토큰을 선택합니다.
    Args:
        model (nn.Module): 훈련된 언어 모델
        start_string (str): 초기 텍스트
        tokenizer: 텍스트 처리를 위한 토크나이저
        device: 생성을 실행할 장치 (cuda/cpu)
        max_length (int): 생성된 시퀀스의 최대 길이
    Returns:
        str: 생성된 텍스트 연속
    """
    # 모델을 평가 모드로 설정
    model.eval()

    # 시작 문자열을 토큰 ID로 변환하고 장치로 이동
    # return_tensors="pt"는 목록 대신 파이토치 텐서를 반환합니다
    tokens = tokenizer.encode(start_string, return_tensors="pt", max_length=max_length, truncation=True).to(device)

    # 입력 토큰으로 생성된 시퀀스 초기화
    generated = tokens

    # 한 번에 하나의 새 토큰 생성
    for _ in range(max_length):
        # 모델의 예측 가져오기
        output = model(generated)                    # 크기: (1, seq_len, vocab_size)
        # 다음 토큰에 대한 로짓 가져오기 (마지막 위치)
        next_token_logits = output[0, -1, :]        # 크기: (vocab_size)
        # 가장 높은 확률을 가진 토큰 선택 (탐욕적 디코딩)
        # 예상 크기와 일치하도록 두 번 확장 (1, 1)
        next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0).unsqueeze(0)
        # 생성된 시퀀스에 새 토큰 추가
        generated = torch.cat((generated, next_token_id), dim=1)
        # 시퀀스 끝 토큰이 생성되면 중지
        if next_token_id.item() == tokenizer.eos_token_id:
            break

    # 토큰 ID를 다시 텍스트로 변환
    generated_text = tokenizer.decode(generated.squeeze().tolist())
    return generated_text

def save_model(model, tokenizer, file_prefix):
    model_state = {
        "state_dict": model.state_dict(),
        "vocab_size": model.vocab_size,
        "emb_dim": model.emb_dim,
        "num_layers": model.num_layers,
        "pad_index": model.pad_index,
        "training": model.training  # 훈련 상태 저장
    }
    torch.save(model_state, f"{file_prefix}_model.pth")
    tokenizer.save_pretrained(f"{file_prefix}_tokenizer")

def load_model(file_prefix):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 먼저 올바른 장치에 상태 딕셔너리 로드
    model_state = torch.load(f"{file_prefix}_model.pth", map_location=device, weights_only=True)
    # 상태 딕셔너리를 로드하기 전에 모델 생성 및 장치 이동
    model = RecurrentLanguageModel(
        model_state["vocab_size"],
        model_state["emb_dim"],
        model_state["num_layers"],
        model_state["pad_index"]
    ).to(device)
    # 모델이 올바른 장치에 있은 후 상태 사전 로드
    model.load_state_dict(model_state["state_dict"])
    # 평가 모드로 설정
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained(f"{file_prefix}_tokenizer")
    return model, tokenizer

def get_hyperparameters():
    """
    모델 훈련을 위한 기본 하이퍼파라미터를 반환합니다.
    Returns:
        tuple: (emb_dim, num_layers, batch_size, learning_rate, num_epochs)
    """
    emb_dim = 128         # 임베딩 차원
    num_layers = 2        # RNN 층 수
    batch_size = 128      # 훈련 배치 크기
    learning_rate = 0.001 # 최적화를 위한 학습률
    num_epochs = 1        # 훈련 에포크 수
    context_size = 30     # 최대 입력 시퀀스 길이
    return emb_dim, num_layers, batch_size, learning_rate, num_epochs, context_size

/home/redinblue/anaconda3/envs/pybuild/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 모델 클래스

RNN 언어 모델 클래스와 초기화 메서드

In [2]:
# ----------------------------
# 순환 언어 모델 클래스
# ----------------------------
def initialize_weights(model):
    """
    다차원 파라미터에는 Xavier 균등 초기화를 사용하고, 편향 및 기타 1차원 파라미터에는
    균등 초기화를 사용하여 모델 가중치를 초기화합니다.
    Args:
        model (nn.Module): 가중치를 초기화해야 하는 파이토치 모델
    """
    # 모델에서 이름을 가진 파라미터를 모두 반복
    for name, param in model.named_parameters():
        # 파라미터가 1차원 이상인 경우 확인 (예: 가중치 행렬)
        if param.dim() > 1:
            # 가중치 행렬에 Xavier 균등 초기화 사용
            # 이는 분산을 일정하게 유지하여 기울기 소실/폭주을 방지하는 데 도움이 됩니다
            nn.init.xavier_uniform_(param)
        else:
            # 1차원 파라미터(편향 등)의 경우 간단한 균등 초기화 사용
            nn.init.uniform_(param)

class ElmanRNNUnit(nn.Module):
    """
    단일 Elman RNN 유닛(간단한 순환 신경망 셀)의 구현입니다.
    입력의 타임스텝 하나를 처리하는 RNN의 기본 구성 요소입니다.
    Args:
        emb_dim (int): 임베딩/은닉 상태 벡터의 차원
    """
    def __init__(self, emb_dim):
        super(ElmanRNNUnit, self).__init__()
        # 은닉-은닉 가중치 행렬: 이전 은닉 상태를 변환
        # 크기: (emb_dim, emb_dim)
        self.Uh = nn.Parameter(torch.rand(emb_dim, emb_dim))
        # 입력-은닉 가중치 행렬: 현재 입력을 변환
        # 크기: (emb_dim, emb_dim)
        self.Wh = nn.Parameter(torch.rand(emb_dim, emb_dim))
        # 변환 합에 더해지는 편향 항
        # 크기: (emb_dim,)
        self.b = nn.Parameter(torch.rand(emb_dim))

    def forward(self, x, h):
        """
        RNN 유닛의 한 타임스텝을 계산합니다.
        Args:
            x (torch.Tensor): 크기가 (batch_size, emb_dim)인 현재 입력 텐서
            h (torch.Tensor): 크기가 (batch_size, emb_dim)인 이전 은닉 상태
        Returns:
            torch.Tensor: 크기가 (batch_size, emb_dim)인 새 은닉 상태
        구현된 공식: h_new = tanh(x @ Wh + h @ Uh + b)
        여기서 @는 행렬 곱셈을 나타냅니다
        """
        # 1. 현재 입력 변환: x @ Wh
        input_transform = x @ self.Wh
        # 2. 이전 은닉 상태 변환: h @ Uh
        hidden_transform = h @ self.Uh
        # 3. 두 변환과 편향을 더함
        # 4. tanh 활성화 함수를 적용하여 새 은닉 상태 얻기
        # tanh는 값을 (-1, 1) 범위로 압축하여 기울기 폭주를 방지하는 데 도움이 됩니다
        return torch.tanh(input_transform + hidden_transform + self.b)

class ElmanRNN(nn.Module):
    """
    전체 시퀀스를 처리하는 다층 Elman RNN 구현입니다.
    더 복잡한 패턴을 학습할 수 있는 더 깊은 네트워크를 만들기 위해 여러 RNN 유닛을 쌓습니다.
    Args:
        emb_dim (int): 임베딩 및 은닉 상태의 차원
        num_layers (int): 쌓인 RNN 층의 수
    """
    def __init__(self, emb_dim, num_layers):
        super().__init__()
        self.emb_dim = emb_dim
        self.num_layers = num_layers
        # 각 층에 대한 RNN 유닛 리스트 생성
        # ModuleList를 사용하여 파이토치가 모든 파라미터를 추적하도록 함
        self.rnn_units = nn.ModuleList(
            [ElmanRNNUnit(emb_dim) for _ in range(num_layers)]
        )

    def forward(self, x):
        """
        모든 RNN 층을 통해 입력 시퀀스를 처리합니다.
        Args:
            x (torch.Tensor): 크기가 (batch_size, seq_len, emb_dim)인 입력 텐서
        Returns:
            torch.Tensor: 크기가 (batch_size, seq_len, emb_dim)인 출력 텐서
        """
        # 입력 텐서에서 차원 가져오기
        batch_size, seq_len, emb_dim = x.size()
        # 각 층의 은닉 상태를 0으로 초기화
        # 각 은닉 상태는 (batch_size, emb_dim) 크기를 가짐
        h_prev = [
            torch.zeros(batch_size, emb_dim, device=x.device)
            for _ in range(self.num_layers)
        ]
        # 각 타임스텝의 출력을 저장할 변수
        outputs = []
        # 각 타임스텝 처리
        for t in range(seq_len):
            # 현재 타임스텝의 입력 가져오기
            input_t = x[:, t]
            # 각 층을 통해 처리
            for l, rnn_unit in enumerate(self.rnn_units):
                # 이 층의 새 은닉 상태 계산
                h_new = rnn_unit(input_t, h_prev[l])
                # 이 층의 은닉 상태 업데이트
                h_prev[l] = h_new
                # 이 층의 출력이 다음 층의 입력이 됨
                input_t = h_new
            # 최종 층의 출력을 결과에 추가
            outputs.append(input_t)
        # 모든 타임스텝의 출력을 단일 텐서로 쌓기
        # 크기: (batch_size, seq_len, emb_dim)
        return torch.stack(outputs, dim=1)

class RecurrentLanguageModel(nn.Module):
    """
    임베딩 층, 다층 RNN 및 출력 프로젝션 층을 결합한 완전한 언어 모델 구현입니다.
    모델 아키텍처는 다음과 같습니다:
    1. 입력 토큰 -> 임베딩 층 -> 임베딩 벡터
    2. 임베딩 벡터 -> RNN 층 -> 문맥 벡터
    3. 문맥 벡터 -> 선형 층 -> 어휘 예측
    Args:
        vocab_size (int): 어휘의 크기 (고유 토큰 수)
        emb_dim (int): 임베딩 및 은닉 상태의 차원
        num_layers (int): RNN 층의 수
        pad_index (int): 패딩 토큰에 사용되는 인덱스
    """
    def __init__(self, vocab_size, emb_dim, num_layers, pad_index):
        super().__init__()
        # 모델 파라미터 저장
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.num_layers = num_layers
        self.pad_index = pad_index

        # 임베딩 층: 토큰 인덱스를 밀집 벡터로 변환
        # pad_index 토큰은 0 벡터로 매핑됩니다
        self.embedding = nn.Embedding(vocab_size, emb_dim, pad_index)

        # 시퀀스 처리를 위한 RNN 층
        self.rnn = ElmanRNN(
            emb_dim=emb_dim,
            num_layers=num_layers
        )

        # RNN 출력을 어휘 예측으로 변환하는 최종 선형 층
        # 출력 크기는 각 가능한 토큰에 대한 로짓을 얻기 위해 vocab_size입니다
        self.fc = nn.Linear(emb_dim, vocab_size)

    def forward(self, x):
        """
        전체 모델을 통해 입력 시퀀스를 처리합니다.
        Args:
            x (torch.Tensor): 토큰 인덱스의 입력 텐서
                            크기: (batch_size, seq_len)
        Returns:
            torch.Tensor: 다음 토큰 예측을 위한 출력 로짓
                         크기: (batch_size, seq_len, vocab_size)
        처리 과정:
        1. 토큰 인덱스를 임베딩으로 변환
        2. 임베딩을 RNN 층을 통해 처리
        3. RNN 출력을 어휘 크기로 프로젝션
        """
        # 토큰 인덱스를 임베딩으로 변환
        # 크기: (batch_size, seq_len) -> (batch_size, seq_len, emb_dim)
        embeddings = self.embedding(x)

        # RNN 층을 통해 처리
        # 크기: (batch_size, seq_len, emb_dim) -> (batch_size, seq_len, emb_dim)
        rnn_output = self.rnn(embeddings)

        # 어휘 크기로 프로젝션하여 로짓 얻기
        # 크기: (batch_size, seq_len, emb_dim) -> (batch_size, seq_len, vocab_size)
        logits = self.fc(rnn_output)

        return logits

## 언어 모델 훈련하기

다음 셀에서 데이터를 로드하고, 모델을 훈련 및 저장합니다.

In [3]:
# ----------------------------
# 순환 신경망 언어 모델을 위한 메인 훈련 루프
# 이 스크립트는 데이터 로딩, 모델 훈련, 검증 및 텍스트 생성을 포함한
# 전체 훈련 과정을 처리합니다.
# ----------------------------
# 재현 가능한 결과를 보장하기 위해 난수 시드 초기화
set_seed(42)

# CUDA 지원 GPU가 있는지 확인하고 그에 따라 장치를 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 설정에서 모델 아키텍처 및 훈련 하이퍼파라미터 추출
# emb_dim: 토큰 임베딩 및 은닉 상태의 차원
# num_layers: 모델의 순환 층 수
# batch_size: 미니 배치 크기
# learning_rate: 옵티마이저 업데이트를 위한 단계 크기
# num_epochs: 훈련 데이터셋을 완전히 통과하는 횟수
# context_size: 최대 입력 시퀀스 길이
emb_dim, num_layers, batch_size, learning_rate, num_epochs, context_size = get_hyperparameters()

# 마이크로소프트의 Phi-3.5-mini 모델을 사용하여 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

# 모델이 처리해야 할 어휘사전의 크기 가져오기
vocab_size = len(tokenizer)

# 뉴스 데이터셋 다운로드 및 훈련과 테스트를 위한 DataLoader 객체 생성
# DataLoader는 배칭 및 셔플링을 처리합니다
data_url = "https://www.thelmbook.com/data/news"
train_dataloader, test_dataloader = download_and_prepare_data(data_url, batch_size, tokenizer, context_size)

# 지정된 아키텍처 매개변수로 RNN 언어 모델 초기화
# vocab_size: 출력 층 차원 결정
# emb_dim: 단어 임베딩 및 은닉 상태의 크기
# num_layers: RNN 층 개수
# pad_token_id: 더 짧은 시퀀스에 대한 패딩에 사용되는 특수 토큰 ID
model = RecurrentLanguageModel(vocab_size, emb_dim, num_layers, tokenizer.pad_token_id)

# 사용 가능한 경우 모델을 GPU로 이동
model.to(device)

# 사용자 지정 초기화 방식을 사용하여 모델 가중치 초기화
# 이는 심층 신경망의 안정적인 훈련에 중요합니다
initialize_weights(model)

# 모델의 총 훈련 가능한 매개변수 수 계산 및 출력
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"총 훈련 가능한 매개변수: {total_params}\n")

# 훈련을 위한 손실 함수(교차 엔트로피) 초기화
# ignore_index=pad_token_id는 패딩 토큰이 손실에 기여하지 않도록 보장합니다
# 이는 모델이 패딩 토큰을 예측하는 것을 학습하는 것을 방지합니다
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# 지정된 학습률로 AdamW 옵티마이저 초기화
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 평가 간격 설정(검증을 수행할 예시 수)
# 200,000 예시는 훈련 시간과 모니터링 빈도 사이의 좋은 균형을 제공합니다
eval_interval = 200_000
examples_processed = 0  # 다음 평가까지의 진행 상황을 추적하기 위한 카운터

# 평가 중에 샘플 텍스트를 생성하기 위한 테스트 컨텍스트 정의
contexts = [
    "Moscow",
    "New York",
    "A hurricane",
    "The President"
]

# 메인 훈련 루프 - 지정된 수의 에포크 반복
for epoch in range(num_epochs):
    # 모델을 훈련 모드로 설정
    model.train()
    print(f"에포크 {epoch+1}/{num_epochs} 시작, 모델 훈련 모드: {model.training}")

    # 이 에포크를 위한 추적 변수 초기화
    total_loss = 0.0      # 모든 배치에 대한 손실 누적
    total_tokens = 0      # 처리된 실제 토큰 수 카운터 (패딩 제외)

    # 훈련 진행 상황 모니터링을 위한 진행률 표시줄 생성
    progress_bar = tqdm(train_dataloader, desc=f"에포크 {epoch+1}/{num_epochs}")

    # 훈련 데이터의 배치 반복
    for batch_idx, (input_seq, target_seq) in enumerate(progress_bar):
        # 사용 가능한 경우 입력 및 타깃 시퀀스를 GPU로 이동
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)

        # 재구성 작업을 위한 현재 배치 차원 가져오기
        batch_size_current, seq_len = input_seq.shape

        # 이전 배치의 그레이디언트 지우기
        # 이는 파이토치가 기본적으로 그레이디언트를 누적하기 때문에 필요합니다
        optimizer.zero_grad()

        # 포워드 패스: 이 배치에 대한 모델 예측 가져오기
        # 출력 크기: (batch_size, seq_len, vocab_size)
        output = model(input_seq)

        # 손실 계산을 위해 출력 및 타깃 텐서 재구성
        # - 출력: CrossEntropyLoss를 위해 (batch_size * seq_len, vocab_size)로 재구성
        # - 타깃: CrossEntropyLoss 요구 사항과 일치하도록 (batch_size * seq_len)로 재구성
        output = output.reshape(batch_size_current * seq_len, vocab_size)
        target = target_seq.reshape(batch_size_current * seq_len)

        # 타깃에서 패딩이 아닌 토큰 수 계산
        # 이는 여러 배치에 대한 평균 손실을 계산하기 위해 총 손실을 이 배치들의 토큰 수로 나누어야 하지만,
        # criterion(output, target)은 배치당 토큰당 평균 손실을 반환하기 때문에 필요합니다.
        # 따라서 배치당 손실을 얻기 위해 토큰당 손실에 토큰 수를 곱할 것입니다
        non_padding_token_count = (target != tokenizer.pad_token_id).sum().item()

        # 모델 예측과 실제 타깃 간의 손실 계산
        loss = criterion(output, target)

        # 백워드 패스: 모델 매개변수에 대한 손실의 그레이디언트 계산
        loss.backward()

        # 계산된 그레이디언트를 사용하여 모델 매개변수 업데이트
        optimizer.step()

        # 이 배치에 대한 실제 손실 값 계산
        # 배치의 총 손실을 얻기 위해 패딩이 아닌 토큰 수에 토큰당 손실을 곱합니다
        loss_value = loss.item() * non_padding_token_count

        # 에포크 통계를 위한 총 손실 누적
        total_loss += loss_value

        # 처리된 실제 토큰 총수 누적
        total_tokens += non_padding_token_count

        # 처리된 샘플 수 카운터 증가
        examples_processed += batch_size_current

        # 현재 배치 손실로 진행률 표시줄 업데이트
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

        # 지정된 수의 샘플을 처리한 후 주기적 평가
        if examples_processed >= eval_interval:
            # 마지막 eval_interval 예시에 대한 평균 손실 계산
            avg_loss = total_loss / total_tokens
            print(f"\n{examples_processed} 샘플 처리 후, 평균 손실: {avg_loss:.4f}")

            # 평가 모드로 전환
            model.eval()

            # 검증 메트릭 계산
            # average_loss: 검증 세트의 평균 손실
            # perplexity: 평균 손실의 지수, 낮을수록 좋음
            # sentences_processed: 평가된 검증 시퀀스 수
            average_loss, perplexity = compute_loss_and_perplexity(
                model, test_dataloader, tokenizer, criterion, device, max_sentences=1000
            )
            print(f"검증 평균 손실: {average_loss:.4f}, 혼잡도: {perplexity:.2f}")

            # 모델 성능을 정성적으로 평가하기 위해 샘플 텍스트 생성
            print("generate_text를 사용하여 문맥을 기반으로 텍스트 생성:\n")
            for context in contexts:
                # 각 테스트 문맥의 뒤를 잇는 텍스트 생성
                generated_text = generate_text(
                    model=model,          # 로드된 언어 모델
                    start_string=context, # 시작 문맥
                    tokenizer=tokenizer,  # 텍스트 변환을 위한 토크나이저
                    device=device,        # CPU 또는 GPU 장치
                    max_length=50         # 생성된 시퀀스의 최대 길이
                )
                print(f"\n문맥: {context}")
                print(f"\n생성된 텍스트: {generated_text}")

            # 계속 훈련하기 위해 훈련 모드로 다시 전환
            model.train()

            # 다음 평가 간격을 위해 카운터 재설정
            examples_processed = 0
            total_loss = 0.0
            total_tokens = 0

    # 에포크 종료 보고
    if total_tokens > 0:
        # 에포크에 대한 평균 손실 계산 및 표시
        avg_loss = total_loss / total_tokens
        print(f"\n에포크 {epoch+1}/{num_epochs}, 평균 손실: {avg_loss:.4f}")
    else:
        # 처리된 토큰이 없는 경우 처리
        print(f"\n에포크 {epoch+1}/{num_epochs} 완료.")

    # 에포크 종료 검증 수행
    model.eval()
    average_loss, perplexity = compute_loss_and_perplexity(
        model, test_dataloader, tokenizer, criterion, device, max_sentences=1000
    )
    print(f"검증 평균 손실: {average_loss:.4f}, 혼잡도: {perplexity:.2f}\n")

    print("generate_text를 사용하여 문맥을 기반으로 텍스트 생성:\n")
    for context in contexts:
        # 각 테스트 문맥의 뒤를 잇는 텍스트 생성
        generated_text = generate_text(
            model=model,
            start_string=context,
            tokenizer=tokenizer,
            device=device,
            max_length=50  # 간결성을 위해 생성을 50 토큰으로 제한
        )
        print(f"\n문맥: {context}")
        print(f"\n생성된 텍스트: {generated_text}")

    # 다음 에포크를 위해 훈련 모드로 재설정
    model.train()

# 나중에 사용하기 위해 훈련된 모델과 토크나이저 저장
# 여기에는 모델 아키텍처, 가중치 및 토크나이저 구성이 포함됩니다
model_name = "RNN_LM"
save_model(model, tokenizer, model_name)

https://www.thelmbook.com/data/news에서 데이터셋 다운로드 중...
다운로드가 완료되었습니다.

아카이브 내용 나열:
아카이브 멤버: news
아카이브 멤버: news/train.txt
아카이브 멤버: news/test.txt

파일 추출 중...
추출이 완료되었습니다.
news/train.txt에서 문장 수 계산 중...
news/train.txt에서 22034911개의 문장을 찾았습니다.
news/test.txt에서 문장 수 계산 중...
news/test.txt에서 449693개의 문장을 찾았습니다.
훈련 문장 수: 22034911
테스트 문장 수: 449693
총 훈련 가능한 매개변수: 8292619

에포크 1/1 시작, 모델 훈련 모드: True


에포크 1/1:   1%|          | 1561/172148 [01:09<2:17:46, 20.64it/s, loss=5.7887]


200064 샘플 처리 후, 평균 손실: 6.3468


검증 평균 손실: 5.5882, 혼잡도: 267.25
generate_text를 사용하여 문맥을 기반으로 텍스트 생성:


문맥: Moscow

생성된 텍스트: Moscow 's mother , the ##-year-old , who was a new-up , the ##-year-old , who was a new-up , the ##-year-old , who was a new-up , and the first time

문맥: New York

생성된 텍스트: New York , the ##-year-old , who was a new-up , the ##-year-old , who was a new-up , the ##-year-old , who was a new-up , and the first time of the ##

문맥: A hurricane

생성된 텍스트: A hurricane , the ##-year-old , who was a new-up , the ##-year-old , who was a new-up , the ##-year-old , who was a new-up , and the first time of the ##


에포크 1/1:   1%|          | 1566/172148 [01:11<8:59:51,  5.27it/s, loss=5.7407] 

...

검증 평균 손실: 4.3992, 혼잡도: 81.39
generate_text를 사용하여 문맥을 기반으로 텍스트 생성:


문맥: Moscow

생성된 텍스트: Moscow has been a major part of the country 's largest city . '' '' said the company 's decision to be a `` very important '' . '' '' . ' '' The ##-year-old was arrested in #### . ' '' The ##-

문맥: New York

생성된 텍스트: New York City Council has been a `` very important '' in the UK . '' '' the former presidential candidate , the former president of the ####s . 's . ' '' The ##-year-old was arrested in #### . ' '' The ##-

문맥: A hurricane

생성된 텍스트: A hurricane , which is the most popularity of the world , is now a major part of the country 's largest city . '' '' said the report . 's . ' '' The ##-year-old was arrested in #### . ' '' The ##


에포크 1/1:  27%|██▋       | 46893/172148 [34:05<5:23:07,  6.46it/s, loss=4.4558]


문맥: The President

생성된 텍스트: The President 's government has been a `` very important '' . '' '' he said . ' . ' '' The ##-year-old was arrested in #### , but the ##-year-old was arrested in #### . ' '' The ##-year-


에포크 1/1:  28%|██▊       | 48248/172148 [35:04<1:30:03, 22.93it/s, loss=4.3283]


KeyboardInterrupt: 

## 모델 테스트하기

아래 셀에서 훈련된 모델을 로드하여 텍스트를 생성합니다.

In [5]:
model_name = "RNN_LM"

# 디스크에서 이전에 저장된 모델과 토크나이저 로드
# 이는 훈련 후 정확한 모델 상태를 재생성합니다
model, tokenizer = load_model(model_name)

model.eval()

average_loss, perplexity = compute_loss_and_perplexity(
    model, test_dataloader, tokenizer, criterion, device, max_sentences=1000
)
print(f"검증 평균 손실: {average_loss:.4f}, 퍼플렉시티: {perplexity:.2f}\n")

# 테스트 섹션의 헤더 출력
print("모델 테스트:\n")

# 모델 성능을 평가하기 위한 테스트 프롬프트 리스트
contexts = [
    "Moscow",
    "New York",
    "A hurricane",
    "The President"
]

# 각 테스트 프롬프트를 반복하고 텍스트 생성
for context in contexts:
    # 탐욕적 디코딩(가장 가능성 있는 토큰)을 사용하여 텍스트 생성
    generated_text = generate_text(
        model=model,          # 로드된 언어 모델
        start_string=context, # 시작 문맥
        tokenizer=tokenizer,  # 텍스트 변환을 위한 토크나이저
        device=device,        # CPU 또는 GPU 장치
        max_length=50         # 생성된 시퀀스의 최대 길이
    )

    # 원본 프롬프트와 모델의 응답 출력
    print(f"\n프롬프트: {context}")
    print(f"\n생성된 응답: {generated_text}")

검증 평균 손실: 4.2843, 퍼플렉시티: 72.55

모델 테스트:


프롬프트: Moscow

생성된 응답: Moscow has been a major part of the country 's largest city of the country . '' 'We 're not going to be a good thing . '' ' I 'm not going to be a good player . '' ) . '' ' I 'm

프롬프트: New York

생성된 응답: New York 's first-time winner of the ##-year-old was a former team-mate in the second half . '' 'The .##-caliber rifle , a spokesman for the ##-year-old boy , who was

프롬프트: A hurricane

생성된 응답: A hurricane season is a huge amount of theft . '' 'The .##-caliber rifle , a spokesman for the Ministry of Justice . ' `` I 'm not going to be a good person . '' ' I 'm not going

프롬프트: The President

생성된 응답: The President 's office said the government had not been a `` significant '' . ' '' It 's not the case . ' '' It was a `` unacceptable '' . ' '' It 's not the case . ' '' It was a `` un
